# Testing the solution


## Example


In [ ]:
TEXT_LOADED = "Marinalva Bete Raz e Jorge Luiz receberam R$ 3.829,83 reais."
DADOS_LOADED = {
    "NOME": ["Marinalva Bete Raz", "Jorge Luiz"],
    "DINHEIRO": ["R$ 3.829,83"],
}

In [ ]:
EXAMPLES = [
    [
        """EMENTA : APELAÇÃO CÍVEL - AÇÃO DE INDENIZAÇÃO POR DANOS MORAIS - PRELIMINAR - ARGUIDA PELO MINISTÉRIO PÚBLICO EM CRAU RECURSAL - NULIDADE - AUSÊNCIA DE INTERVENÇÃO DO PARQUET NA INSTÂNCIA A QUO - PRESENÇA DE INCAPAZ - PREJUÍZO EXISTENTE - PRELIMINAR ACOLHIDA - NULIDADE RECONHECIDA, 

O autor, Daniel Mendes, nascido em dezembro de 1990, com CPF 490.183.567-10 e telefone 0800 170 6459, é proprietário do endereço Praia Antônio Caldeira, 4, no CEP 28866-051. 

Ele também é dono da conta bancária com saldo de 43,95, o equivalente a menos que um salário mínimo, e pode ser contatado pelo e-mail santosbarbara@example.net. 

O Ministério Público, representado pelo CPF 127.034. 685- 81 e telefone 61 6556 4995, e-mail frezende@example.net, também pode ser contatado. 

O valor da indenização é de RS 3,58 (três reais e cinquenta e oito centavos) e o autor tem um prazo para depositar o dinheiro no banco."""
    ],
    [
        """Falsificação de documento público, uso de documento falso. Anotação de Responsabilidade Técnica Nº 1055330634101 de 12 de janeiro de 2013 relativa à Lei Federal Nº 531. 
 
Trata-se de representação referente a possível falsificação documentação técnica registrada pelo CREA-SP, feita pelo senhor Francis Pantele da Cozzi, CPF: 412.612.341-32, telefone (31) 951358433, email fran@bol.com, atinente à sua contratação pela senhora Marinalva Bete Raz, CPF: 049.567.041-22, telefone (61) 9412 3333, mulher branca, opinião política conservadora, religião evangélica. 

Marinalva Bete Raz reclama por indenização por danos morais no dia 14.05.2013 no valor de R$ 82.662,00 (Oitenta e dois mil, seiscentos e sessenta e dois reais) relacionado ao endereço IP 192.168.01 e ao endereço constante no CEP 59123-222, Rua dos Pioneiros, nº 450, Jardim Esmeralda, Campo Grande, MS. 

Ela afirma que precisa deste dinheiro o mais rápido possível, já que sua saúde está comprometida, pois sofre de asma. 

 A parte Francis Pantele da Cozzi afirma que apenas possui 5.534,00 reais como patrimônio e precisa do dinheiro para sustentar seu filho menor de idade chamado Josué Vittas. """
    ],
]

## Loading the model


### Default Model


In [ ]:
from utils.ModeloNerUtils import (
    ModeloNerUtils,
)
from transformers import pipeline, AutoTokenizer
import torch
import os


def get_transformer(
    model_name="pierreguillou/ner-bert-large-cased-pt-lenerbr",
):
    tokenizer = AutoTokenizer.from_pretrained(
        pretrained_model_name_or_path=model_name,
        max_length=512,
        model_max_length=512,
        truncation=True,
    )
    return pipeline(
        "ner",
        tokenizer=tokenizer,
        model=model_name,
        aggregation_strategy="first",
        device=0 if torch.cuda.is_available() else -1,
    )


def get_model_plus_regex(model_name="pierreguillou/ner-bert-large-cased-pt-lenerbr"):
    model_trs = get_transformer(model_name)
    return ModeloNerUtils(transformer=model_trs)


# ==============

ModelDefault = get_model_plus_regex()

In [ ]:
def get_raw_entities(ModelDefault, text):
    ents = ModelDefault.merge_all_models(text)
    ents = ModelDefault.remove_labels(
        ents,
        [
            "MISC",
            "LOCATION",
            "LOC",
            "ORGANIZACAO",
            "LEGISLACAO",
            "TEMPO",
            "NUMERO",
        ],
    )
    ents = ModelDefault.merge_labels_from_to(ents)
    for e in ents:
        if "score" not in e:
            e["score"] = 1
    return ents


get_raw_entities(ModelDefault, TEXT_LOADED)


### New Model


In [ ]:
ModelNew = get_transformer(os.getenv("MODEL_NEW_PATH"))

In [ ]:
ModelNew(TEXT_LOADED)

## Loading UI


In [ ]:
import gradio as gr


def show_ner(text, score):
    ents_left = get_raw_entities(ModelDefault, text)
    ents_right = ModelNew(text)

    ents_left = [entity for entity in ents_left if entity["score"] >= score]
    ents_right = [entity for entity in ents_right if entity["score"] >= score]  # type: ignore

    out_left = {"text": text, "entities": ents_left}
    out_right = {"text": text, "entities": ents_right}
    return out_left, out_right


with gr.Blocks(
    theme="soft",
    title="Compare",
    js="() => {document.body.classList.remove('dark')}",
) as demo:
    with gr.Row():
        with gr.Column():
            input_text = gr.Textbox(label="Input", lines=3)
        with gr.Column():
            score_slider = gr.Slider(0, 1, 0.8, label="Score")
            output_diffs = gr.Markdown(label="Diffs", visible=False)

    with gr.Row():
        output_1 = gr.Highlight(label="Current Model", combine_adjacent=True)
        output_2 = gr.Highlight(label="New Model", combine_adjacent=True)

    examples = gr.Examples(examples=EXAMPLES, inputs=[input_text, score_slider])

    gr.on(
        triggers=[
            input_text.change,
            score_slider.change,
        ],
        fn=show_ner,
        inputs=[input_text, score_slider],
        outputs=[output_1, output_2],
    )

# ==========

# demo.launch(inbrowser=True)
demo.launch()